In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
!pip list 

Package                Version
---------------------- -----------
absl-py                0.9.0
asn1crypto             0.24.0
astunparse             1.6.3
attrs                  19.3.0
backcall               0.1.0
bleach                 3.1.5
cachetools             4.1.0
certifi                2020.4.5.1
chardet                3.0.4
cryptography           2.1.4
cycler                 0.10.0
decorator              4.4.2
defusedxml             0.6.0
entrypoints            0.3
gast                   0.3.3
google-auth            1.14.2
google-auth-oauthlib   0.4.1
google-pasta           0.2.0
grpcio                 1.28.1
h5py                   2.10.0
idna                   2.6
importlib-metadata     1.6.0
ipykernel              5.1.1
ipython                7.14.0
ipython-genutils       0.2.0
ipywidgets             7.5.1
JayDeBeApi             1.2.2
jedi                   0.17.0
Jinja2                 2.11.2
joblib                 0.15.1
JPype1                 0.7.5
jsonschema             3

In [14]:
### 1. Set environment variables, if necessary
#import os
#os.environ['JAVA_HOME']='C:\Progra~1\Java\jdk1.8.0_241'
#os.environ['CLASSPATH'] = 'C:\interSystems\IRIS20194\dev\java\lib\JDK18\intersystems-jdbc-3.0.0.jar'
#os.environ['HADOOP_HOME']='C:\hadoop\bin'  #winutil binary must be in Hadoop's Home

In [3]:
### 2. Get jdbc connection and cursor
import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:51773/USER"  #We can use Docker mahine's port: url = "jdbc:IRIS://172.17.0.1:8091/USER"
driver = 'com.intersystems.jdbc.IRISDriver'
user = "SUPERUSER"  
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "/tf/intersystems-jdbc-3.1.0.jar"


In [4]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

In [30]:
### 3. specify the source data table
dataTable = 'DataMining.IrisDataset'

In [31]:
### 4. Get the result and display
curs.execute("select TOP 20 * from %s" % dataTable)
result = curs.fetchall()
print("Total records: " + str(len(result)))
for i in range(len(result)):
    print(result[i])

Total records: 20
(1, 1.4, 0.2, 5.1, 3.5, 'Iris-setosa')
(2, 1.4, 0.2, 4.9, 3.0, 'Iris-setosa')
(3, 1.3, 0.2, 4.7, 3.2, 'Iris-setosa')
(4, 1.5, 0.2, 4.6, 3.1, 'Iris-setosa')
(5, 1.4, 0.2, 5.0, 3.6, 'Iris-setosa')
(6, 1.7, 0.4, 5.4, 3.9, 'Iris-setosa')
(7, 1.4, 0.3, 4.6, 3.4, 'Iris-setosa')
(8, 1.5, 0.2, 5.0, 3.4, 'Iris-setosa')
(9, 1.4, 0.2, 4.4, 2.9, 'Iris-setosa')
(10, 1.5, 0.1, 4.9, 3.1, 'Iris-setosa')
(11, 1.5, 0.2, 5.4, 3.7, 'Iris-setosa')
(12, 1.6, 0.2, 4.8, 3.4, 'Iris-setosa')
(13, 1.4, 0.1, 4.8, 3.0, 'Iris-setosa')
(14, 1.1, 0.1, 4.3, 3.0, 'Iris-setosa')
(15, 1.2, 0.2, 5.8, 4.0, 'Iris-setosa')
(16, 1.5, 0.4, 5.7, 4.4, 'Iris-setosa')
(17, 1.3, 0.4, 5.4, 3.9, 'Iris-setosa')
(19, 1.7, 0.3, 5.7, 3.8, 'Iris-setosa')
(20, 1.5, 0.3, 5.1, 3.8, 'Iris-setosa')
(21, 1.7, 0.2, 5.4, 3.4, 'Iris-setosa')


In [32]:
### 5. CLose and clean - I keep them open for next accesses.
#curs.close()
#conn.close()

In [33]:
### transform SQL results "sqlData"to Pandas dataframe "df", then further to NumPy array "arrayN" for further ML pipelines 
import pandas as pd
sqlData = "SELECT * from DataMining.IrisDataset"
df= pd.io.sql.read_sql(sqlData, conn)
df = df.drop('ID', 1)
df = df[['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']]
# set the labels to 0, 1, 2, for NumPy matrix
df.replace('Iris-setosa', 0, inplace=True)
df.replace('Iris-versicolor', 1, inplace=True)
df.replace('Iris-virginica', 2, inplace=True)
# turn dataframe into Numpy array
arrayN = df.to_numpy()
### 6. CLose and clean - if connection is not needed anymore?
#curs.close()
#conn.close()

In [34]:
df.head(5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,4.9,3.1,1.5,0.1,0
1,4.4,3.0,1.3,0.2,0
2,5.1,3.4,1.5,0.2,0
3,5.0,3.5,1.3,0.3,0
4,4.5,2.3,1.3,0.3,0


In [35]:
df.describe()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,1.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [36]:
import numpy as np 
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

In [37]:
# keep e.g. 20% = 30 rows as test data; trained on another e.g. 80% = 120 rows
X = arrayN[:,0:4]
y = arrayN[:,4]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)
# make 80% of random rows into a Train set
labels1 = np.reshape(Y_train,(120,1))
train = np.concatenate([X_train, labels1],axis=-1)
# make 20% of left rows into Test set
lTest1 = np.reshape(Y_validation,(30,1))
test = np.concatenate([X_validation, lTest1],axis=-1)
# write the train data set into a Pandas frame
dfTrain = pd.DataFrame({'SepalLength':train[:, 0], 'SepalWidth':train[:, 1], 'PetalLength':train[:, 2], 'PetalWidth':train[:, 3], 'Species':train[:, 4]})
dfTrain['Species'].replace(0, 'Iris-setosa', inplace=True)
dfTrain['Species'].replace(1, 'Iris-versicolor', inplace=True)
dfTrain['Species'].replace(2, 'Iris-virginica', inplace=True)
# write the test data into another Pandas frame
dfTest = pd.DataFrame({'SepalLength':test[:, 0], 'SepalWidth':test[:, 1], 'PetalLength':test[:, 2], 'PetalWidth':test[:, 3], 'Species':test[:, 4]})
dfTest['Species'].replace(0, 'Iris-setosa', inplace=True)
dfTest['Species'].replace(1, 'Iris-versicolor', inplace=True)
dfTest['Species'].replace(2, 'Iris-virginica', inplace=True)
### 3. specify temp table names
#dataTable = 'DataMining.IrisDataset'
dtTrain = 'TRAIN03'
dtTest = "TEST03"
### 4. Create 2 temporary tables - you can try drop tables then re-create them every time
curs.execute("Create Table %s (%s DOUBLE, %s DOUBLE, %s DOUBLE, %s DOUBLE, %s VARCHAR(100))" % (dtTrain, dfTrain.columns[0], dfTrain.columns[1], dfTrain.columns[2], dfTrain.columns[3], dfTrain.columns[4]))
curs.execute("Create Table %s (%s DOUBLE, %s DOUBLE, %s DOUBLE, %s DOUBLE, %s VARCHAR(100))" % (dtTest, dfTest.columns[0], dfTest.columns[1], dfTest.columns[2], dfTest.columns[3], dfTest.columns[4]))
### 5. write Train set and Test set into the tales. You can try to delete old record then insert everytime. 
curs.fast_executemany = True
curs.executemany( "INSERT INTO %s (SepalLength, SepalWidth, PetalLength, PetalWidth, Species) VALUES (?, ?, ?, ? ,?)" % dtTrain,
     list(dfTrain.itertuples(index=False, name=None)) )
curs.executemany( "INSERT INTO %s (SepalLength, SepalWidth, PetalLength, PetalWidth, Species) VALUES (?, ?, ?, ? ,?)" % dtTest,
     list(dfTest.itertuples(index=False, name=None)) )
### 6. CLose and clean - if connection is not needed anymore?
#curs.close()
#conn.close()

In [38]:
### 6. CLose and clean - if connection is not needed anymore?
curs.close()
conn.close()